# Julia 深度學習：卷積神經網路模型簡介

## 期末主題 - 手寫辨識


### 本課程範例實際在GPU執行時，失敗無效。

### 解決方式參考@ magikerwin1993的做法如下
#### 參考來源: 
- https://github.com/magikerwin1993/1st-DL-CVMarathon/blob/master/homework/Day-033/julia_033_hw.ipynb
- https://github.com/magikerwin1993/1st-DL-CVMarathon/blob/master/homework/Day-033/julia_033_example.ipynb

In [2]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics
using Flux.Optimise: Optimiser, WeightDecay
using Statistics, Random
import ProgressMeter
import MLDatasets
using CUDAapi

## Set the parameters

In [3]:
η = 1.0e-4             # learning rate
λ = 3.0e-4             # L2 regularizer param, implemented as weight decay
epochs = 100          # number of epochs
batchsize = 128      # batch size

128

# Using CUDA

In [4]:
use_cuda = CUDAapi.has_cuda_gpu()
if use_cuda
    device = gpu
    println("Training on GPU")
else
    device = cpu
    println("Training on CPU")
end

Training on GPU


# Read the datasets

In [5]:
function get_data(batchsize=256)
    xtrain, ytrain = MLDatasets.MNIST.traindata(Float32)
    xtest , ytest  = MLDatasets.MNIST.testdata(Float32)

    xtrain = reshape(xtrain, 28, 28, 1, :)
    xtest  = reshape(xtest, 28, 28, 1, :)

    ytrain = onehotbatch(ytrain, 0:9)
    ytest  = onehotbatch(ytest, 0:9)

    train_loader = DataLoader(xtrain, ytrain, batchsize=batchsize, shuffle=true)
    test_loader  = DataLoader(xtest, ytest,  batchsize=batchsize)
    
    return train_loader, test_loader
end

train_loader, test_loader = get_data(batchsize);

# CNN Model

In [6]:
# write your model here
num_params(model) = sum(length, Flux.params(model))

function buildModel(imgsize=(28,28,1))
    return Chain(
    x -> reshape(x, imgsize..., :),
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    flatten,
    Dense(288, 10),
    softmax)
end

model = buildModel() |> device;
println("CNNs model: $(num_params(model)) trainable params");

CNNs model: 16938 trainable params


In [7]:
fake_input = randn((28, 28, 1)) |> device
fake_output = model(fake_input)

┌ Warning: `Target(triple::String)` is deprecated, use `Target(; triple=triple)` instead.
│   caller = ip:0x0
└ @ Core :-1


10×1 CuArrays.CuArray{Float32,2,Nothing}:
 0.0948608 
 0.06338216
 0.05639725
 0.12702903
 0.1754451 
 0.05412623
 0.10510717
 0.11789113
 0.10662533
 0.09913574

# Loss function

In [8]:
loss(ŷ, y) = logitcrossentropy(ŷ, y)

round4(x) = round(x, digits=4)

function calc_loss_accuracy(loader, model, device)
    l = 0f0
    acc = 0
    ntot = 0
    for (x, y) in loader
        x, y = x |> device, y |> device
        ŷ = model(x)
        l += loss(ŷ, y) * size(x)[end]        
        acc += sum(onecold(ŷ |> cpu) .== onecold(y |> cpu))
        ntot += size(x)[end]
    end
    return (loss = l/ntot |> round4, acc = acc/ntot*100 |> round4)
end

calc_loss_accuracy (generic function with 1 method)

# Callback Function

In [9]:
# callback function
function callback(epoch)
    train = calc_loss_accuracy(train_loader, model, device)
    test = calc_loss_accuracy(test_loader, model, device)        
    println("Epoch: $epoch   Train: $(train)   Test: $(test)")
end

callback (generic function with 1 method)

# Training

In [10]:
# define optimizer
opt = ADAM(η) 
if λ > 0 
    opt = Optimiser(opt, WeightDecay(λ))
end

Optimiser(Any[ADAM(0.0001, (0.9, 0.999), IdDict{Any,Any}()), WeightDecay(0.0003)])

In [11]:
params_model = Flux.params(model)

println("Starting Training")
callback(0)

for epoch in 1:epochs
    progress = ProgressMeter.Progress(length(train_loader))
    
    # update parameters
    for (x, y) in train_loader
        x = x |> device
        y = y |> device
        grads_model = Flux.gradient(params_model) do
            ŷ = model(x)
            loss(ŷ, y)
        end
        Flux.Optimise.update!(opt, params_model, grads_model)
        ProgressMeter.next!(progress)   # comment out for no progress bar
    end
    
    #  logging
    if epoch % 5 == 0
        callback(epoch)
    end
end

Starting Training
Epoch: 0   Train: (loss = 2.3027f0, acc = 9.4283)   Test: (loss = 2.3027f0, acc = 9.6)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:41
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


Epoch: 5   Train: (loss = 1.5636f0, acc = 93.03)   Test: (loss = 1.5591f0, acc = 93.17)

Progress:   3%|██                                       |  ETA: 0:00:04

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


...

Epoch: 90   Train: (loss = 1.5096f0, acc = 96.9267)   Test: (loss = 1.5069f0, acc = 96.97)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


Epoch: 95   Train: (loss = 1.5077f0, acc = 97.0933)   Test: (loss = 1.5045f0, acc = 97.29)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


Epoch: 100   Train: (loss = 1.509f0, acc = 97.0017)   Test: (loss = 1.5066f0, acc = 97.01)


# Evaluation

In [12]:
test = calc_loss_accuracy(test_loader, model, device)        
println("Test: $(test)")

Test: (loss = 1.5066f0, acc = 97.01)
